In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path
import plotly.graph_objects as go

# Fix import
sys.path.append(os.path.abspath(os.path.join('..')))
from facilitators import BarFacilitator, StackedQuantityEvolutionFacilitatorBase

In [2]:
PROJECT_ROOT_DIR = Path().resolve().parent
SOLUTION_DIR = PROJECT_ROOT_DIR / "solution_files"

DAILY_DATA = SOLUTION_DIR / "solution_data_daily_d20_h6_s6_no_invest.sol"
SEASON_DATA = SOLUTION_DIR / "solution_data_season_d20_h6_s6_no_invest.sol"

REGION = "ALL"

In [3]:
bar = BarFacilitator(sol_paths={"daily": DAILY_DATA, "season": SEASON_DATA}, type_of_data_to_read="TotalCapacityAnnual", 
                     x_grouping_columns=["Year", "Source"], legend_grouping_columns=["Technology", "Source"])

data = bar.get_relevant_data()

data.filter_by_list(column="Technology", identifier_list=["D_Battery_Li-Ion"])

if REGION == "ALL":
    data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=REGION, column_to_sum="Value")

data.filter_by_identifier(column="Region", identifier=REGION)

fig = bar.plot(bar.generate_traces(data))

fig.update_layout(
    title="Total Capacity Annual",
    xaxis_title="Year",
    yaxis_title="Capacity (GW)",
    width=1200,
    height=600
    )

fig.show()

No unit conversion applied!


In [4]:
YEAR = 2050

bar = BarFacilitator(sol_paths={"daily": DAILY_DATA, "season": SEASON_DATA}, type_of_data_to_read="RateOfActivity", 
                     x_grouping_columns=["TS", "Source"], legend_grouping_columns=["Technology", "Source"])

data = bar.get_relevant_data()

# Fix yearsplit division
data.df["Value"] *= data.year_split

# Keep only H2
data.filter_by_list(column="Technology", identifier_list=["D_Gas_H2"])

# Keep only modes 1 and 2
data.filter_by_list("Mode", [1, 2])

# Set mode 1 to negative
data.df.loc[data.df["Mode"] == 2, "Value"] *= -1

# Aggregate over Mode
data.aggreagate_all_by_sum(column_to_aggregate="Mode", aggregated_entry_name=-1, column_to_sum="Value")

if REGION == "ALL":
    data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=REGION, column_to_sum="Value")

data.df = data.df[data.df["Year"] == YEAR]

fig = bar.plot(bar.generate_traces(data))

fig.update_layout(
        title=f"Rate of activity, Region: {REGION}, Year: {YEAR}",
        yaxis_title="TWh",
        xaxis_title="Year",
    )

fig.show()

Converting from PetaJoules to TerraWattHours


# Capacity Evolution

In [17]:

def storage_capacity_evolution_fig(sol_path: str, region: str = "ALL", title_addon: str = None) -> go.Figure:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=sol_path, type_of_data_to_read="TotalCapacityAnnual", 
                                                                region=region, extra_identifying_columns=["Technology"])

    d = storage_evolution.get_relevant_data()
    d.filter_by_containing_string(column="Technology", identifier="D_")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    # Remove those who are 0 for all years
    techs_over_zero = d.df[d.df["Value"] > 0]["Technology"].unique()
    d.filter_by_list(column="Technology", identifier_list=techs_over_zero)

    fig = storage_evolution.plot(storage_evolution.generate_traces(d))

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Storage Capacity, Region: {region}" + f", {title_addon}" if title_addon is not None else "",
        yaxis_title="GW",
        xaxis_title="Year",
    )

    return fig


'storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=SEASON_DATA, type_of_data_to_read="TotalCapacityAnnual", \n                                                            region=REGION, extra_identifying_columns=["Technology"])\n\nd = storage_evolution.get_relevant_data()\nd.filter_by_containing_string(column="Technology", identifier="D_")\n\nif REGION == "ALL":\n    d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=REGION, column_to_sum="Value")\n\nfig = storage_evolution.plot(storage_evolution.generate_traces(d))\n\nfig.update_layout(\n    width=1400,\n    height=800,\n    title=f"Storage Capacity, Region: {REGION}, ",\n    yaxis_title="GW",\n    xaxis_title="Year",\n)'

In [18]:
storage_capacity_evolution_fig(DAILY_DATA, region=REGION, title_addon="Daily").show()
storage_capacity_evolution_fig(SEASON_DATA, region=REGION, title_addon="Seasonal").show()

No unit conversion applied!


No unit conversion applied!
